Microsoft Autogen Assistant Chat Example

Overview
This script demonstrates how to build an interactive AI assistant chat using Microsoft Autogen and AgentOps.
You will see how to:
  - Initialize an assistant agent with OpenAI's GPT-4 model
  - Equip the agent with a calculator tool for step-by-step math problem solving
  - Track and log all agent interactions automatically with AgentOps
  - Simulate a conversation loop where the assistant and user exchange messages until the task is complete
This approach is useful for building conversational AI agents that can reason through complex tasks, show their work, and provide transparent, auditable results.


In [ ]:
# AgentOps ensures all actions are tracked, making it easy to monitor, debug, and analyze your agent's performance in real time.
#
# First let's install the required packages
%pip install -U ag2[autogen-agentchat]
%pip install -U "autogen-ext[openai]"
%pip install -U agentops
%pip install -U python-dotenv
%pip install -U nest_asyncio
# Then import them

In [ ]:
import os
import asyncio
from dotenv import load_dotenv
from IPython.core.error import (
    StdinNotImplementedError,
)
from typing import Annotated, Literal
import agentops
import nest_asyncio
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

Next, we'll set our API keys. There are several ways to do this, the code below is just the most foolproof way for the purposes of this notebook. It accounts for both users who use environment variables and those who just want to set the API Key here in this notebook.

[Get an AgentOps API key](https://agentops.ai/settings/projects)

1. Create an environment variable in a .env file or other method. By default, the AgentOps `init()` function will look for an environment variable named `AGENTOPS_API_KEY`. Or...

2. Replace `<your_agentops_key>` below and pass in the optional `api_key` parameter to the AgentOps `init(api_key=...)` function. Remember not to commit your API key to a public repo!

In [ ]:
load_dotenv()
os.environ["AGENTOPS_API_KEY"] = os.getenv("AGENTOPS_API_KEY", "your_api_key_here")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "your_openai_api_key_here")
# Ensure API key is available
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable is required")
# Define model and API key
model_name = "gpt-4-turbo"  # Or "gpt-4o" / "gpt-4o-mini" as per migration guide examples
# Create the model client
model_client = OpenAIChatCompletionClient(model=model_name, api_key=openai_api_key, seed=42, temperature=0)
# When initializing AgentOps, you can pass in optional tags to help filter sessions
agentops.init(trace_name="autogen-agent-chat", tags=["autogen-agent-chat", "agentops-example"])

In [ ]:
# Define a simple calculator tool the assistant can use
Operator = Literal["+", "-", "*", "/"]
def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

In [ ]:
# Create the assistant agent with access to the calculator tool
assistant = AssistantAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant with access to a calculator tool. "
    "When given a mathematical expression, you must break it down and use the calculator tool "
    "for ALL arithmetic operations until you reach the final answer. "
    "Do not stop until you have calculated the complete final result. "
    "Show your work step by step and provide the final numerical answer. "
    "Only say 'TERMINATE' after you have completed the entire calculation.",
    model_client=model_client,
    tools=[calculator],
    reflect_on_tool_use=True,
)

In [ ]:
# Create a user proxy agent to represent the human user
user_proxy = UserProxyAgent(
    name="User",
    description="A user proxy agent that represents the human user.",
)

In [ ]:
# Main async function to run the chat loop
async def main():
    # The initial question for the assistant to solve
    initial_message = "What is (1423 - 123) / 3 + (32 + 23) * 5? Please complete the entire calculation step by step and show the final answer."
    print(f"User: {initial_message}")

    # Start the conversation history with the user's question
    conversation_history = [TextMessage(content=initial_message, source="user")]

    max_turns = 10  # Prevent infinite loops
    turn = 0

    # Conversation loop: user and assistant take turns
    while turn < max_turns:
        turn += 1
        print(f"\n--- Turn {turn} ---")

        # Send the conversation so far to the assistant and get a response
        response = await assistant.on_messages(conversation_history, CancellationToken())

        if response.chat_message:
            assistant_message = response.chat_message
            print(f"Assistant: {assistant_message.to_text()}")

            # Check if the assistant has finished the calculation
            message_text = assistant_message.to_text()
            if "TERMINATE" in message_text or "final answer" in message_text.lower():
                print("Calculation completed!")
                break

            # Add assistant's message to the conversation for the next turn
            conversation_history.append(TextMessage(content=message_text, source="assistant"))

            # If not done, prompt the assistant to continue
            if turn < max_turns - 1: 
                continue_prompt = "Please continue with the remaining calculations to get the final answer."
                conversation_history.append(TextMessage(content=continue_prompt, source="user"))
                print(f"User: {continue_prompt}")

        else:
            print("Assistant did not provide a response.")
            break

In [ ]:
nest_asyncio.apply()
asyncio.run(main())

You can view data on this run at [app.agentops.ai](app.agentops.ai).

The dashboard will display LLM events for each message sent by each agent, including those made by the human user.